In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import os
import cv2 as cv
import numpy as np
import pprint
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]
pd.options.display.max_columns = None
pd.options.display.width=None

from config import folder_locs, interest_cols, rando_seed

# Instructions:
1. Put all your photos in the input_photos folder
2. Put the annotation file in the input_csvs folder
3. Set the input_csv_filename variable (in the User Input section below) to the name of the annotation file
4. Run all the cells (top menu: Run > Run All Cells)

# User Input

enter the name of the csv file. 

if the file is an excel file, please save the file as a csv file by:
+ opening the excel file
+ selecting File from the menu at the top
+ selecting Save As
+ selecting CSV UTF-8 from the drop-down menu
+ selecting Save

In [3]:
input_csv_filename = 'groundtruth.csv'

# Code

creating the relative filepath for the annoation file

In [4]:
input_csv_filepath = folder_locs['in_files'] + input_csv_filename
# print(input_csv_filepath)

loading the annotation file as a dataframe

In [5]:
csv_df = pd.read_csv(input_csv_filepath)
csv_df

,# 1: Detection or Track-id,2: Video or Image Identifier,3: Unique Frame Identifier,4-7: Img-bbox(TL_x,TL_y,BR_x,BR_y),8: Detection or Length Confidence,9: Target Length (0 or -1 if invalid),10-11+: Repeated Species,Confidence Pairs or Attributes
0,0,2WDC5729-camera_1-20190225-155821Z(StitchedCli...,0,652,217,1268,449,1,-1,Swordfish,1
1,0,2WDC5729-camera_1-20190225-155821Z(StitchedCli...,1,639,218,1267,451,1,-1,Swordfish,1
2,0,2WDC5729-camera_1-20190225-155821Z(StitchedCli...,2,608,219,1270,452,1,-1,Swordfish,1
3,0,2WDC5729-camera_1-20190225-155821Z(StitchedCli...,3,596,221,1265,456,1,-1,Swordfish,1
4,0,2WDC5729-camera_1-20190225-155821Z(StitchedCli...,4,582,222,1264,458,1,-1,Swordfish,1
...,...,...,...,...,...,...,...,...,...,...,...
248551,176056,trip0001camera1_76_57050.jpeg,123495,506,200,747,269,1,-1,Bigeyetuna,1
248552,176056,trip0001camera1_76_57051.jpeg,123496,508,201,749,270,1,-1,Bigeyetuna,1
248553,176056,trip0001camera1_76_57052.jpeg,123497,504,202,745,271,1,-1,Bigeyetuna,1
248554,176056,trip0001camera1_76_57053.jpeg,123498,500,203,741,272,1,-1,Bigeyetuna,1


In [6]:
cur_photos = os.listdir(folder_locs['in_pics']) 
cur_photos = [x for x in cur_photos if '.jpeg' in x]
cur_photos[:5]

['1045238-camera_1-20191010-041522Z_10428.jpeg',
 '1045238-camera_1-20191010-041522Z_1604.jpeg',
 '1045238-camera_1-20191010-041522Z_1605.jpeg',
 '1045238-camera_1-20191010-041522Z_1606.jpeg',
 '1045238-camera_1-20191010-041522Z_1607.jpeg']

In [7]:
csv_df = csv_df.set_index(interest_cols['filename']).loc[cur_photos].reset_index()
csv_df

,2: Video or Image Identifier,# 1: Detection or Track-id,3: Unique Frame Identifier,4-7: Img-bbox(TL_x,TL_y,BR_x,BR_y),8: Detection or Length Confidence,9: Target Length (0 or -1 if invalid),10-11+: Repeated Species,Confidence Pairs or Attributes
0,1045238-camera_1-20191010-041522Z_10428.jpeg,163464,53560,509,522,593,585,1,-1,Lazyline,1
1,1045238-camera_1-20191010-041522Z_1604.jpeg,163455,53483,418,522,478,588,1,-1,Lazyline,1
2,1045238-camera_1-20191010-041522Z_1605.jpeg,163455,53484,375,527,442,593,1,-1,Lazyline,1
3,1045238-camera_1-20191010-041522Z_1606.jpeg,163455,53485,342,531,405,597,1,-1,Lazyline,1
4,1045238-camera_1-20191010-041522Z_1607.jpeg,163455,53486,310,536,369,602,1,-1,Lazyline,1
...,...,...,...,...,...,...,...,...,...,...,...
40796,WDG2008-camera_2-20191017-150142Z_68395.jpeg,175571,95824,278,329,483,425,1,-1,Escolar,1
40797,WDG2008-camera_2-20191017-150142Z_69944.jpeg,175574,95825,181,507,425,621,1,-1,Bigeyetuna,1
40798,WDG2008-camera_2-20191017-150142Z_69947.jpeg,175575,95826,216,527,456,622,1,-1,Bigeyetuna,1
40799,WDG2008-camera_2-20191017-150142Z_798.jpeg,174825,95215,234,305,405,472,1,-1,Bigeyetuna,1


getting all the uniuqe labels in the annotation file

In [8]:
unique_labels = csv_df[interest_cols['label']].unique()
# print(unique_labels)

generating colors for the bounding boxes of the unique labels.  
a random seed has been set in the config.py file, under the scripts folder.  
this means that the colors generated for a certain annotation file will always be the same

In [9]:
np.random.seed(rando_seed)
rando_bgrs = np.random.choice(a=np.arange(start=30, 
                                          stop=225), 
                              size=3*len(unique_labels), 
                              replace=False).reshape(len(unique_labels), 3).tolist()

creating a dictionary with the unique labels as keys, and the bounding box colors as the values

In [10]:
label_bgrs = {}
for i, a_label in enumerate(unique_labels):
    label_bgrs[a_label] = tuple(rando_bgrs[i])
# pprint.pprint(label_bgrs)

iterate through each file to draw bounding boxes and labels.  
save the images in the output_photos folder.

If you want to see the step-by-step process of the code, uncomment all the 'print' blocks of code.  
please make sure you are working with a small dataset if you are doing this (the sample file contained 5 images). 

In [11]:
txt_color=(255, 255, 255) # the text color will be green
filenames = csv_df[interest_cols['filename']].unique() # get all the unique filenames 
n_files = filenames.shape[0] # get a count of the total number of unique filenames

for i, filename in enumerate(filenames):
    try:
    
        # print(f'\n\nWORKING WITH FILE {i+1} OF {n_files}')
        # print(f'\nFILENAME:\n{filename}')

        cur_df = csv_df[csv_df[interest_cols['filename']]==filename].copy() # subset the dataframe for one photo

        # print(f'\nTHESE ARE ALL THE ANNOTATIONS FOR THE FILE:\n')
        # for i, row in cur_df.iterrows():
        #     print(row)

        cur_filepath = folder_locs['in_pics'] + filename # create the filepath for the filename

        # print(f'\nTHIS IS THE FILEPATH USED TO READ THE IMAGE:\n{cur_filepath}')

        cur_img = cv.imread(cur_filepath) # load the image file

        # print(f'\nTHIS IS THE RAW IMAGE\n')
        # plt.imshow(cv.cvtColor(cur_img, cv.COLOR_BGR2RGB))
        # plt.show()


        line_thickness = max(round(sum(cur_img.shape) / 2 * 0.003), 2)  # calculate bbox line thickness

        # print(f'\nTHIS WILL BE THE LINE THICKNESS OF THE BBOX:\n{line_thickness}')

        # iterate through each row of the subset dataframe, and draw box

        # print(f'\nWE WILL NOW ITERATE THROUGH EACH ANNOTATION')

        n_rows = cur_df.shape[0]
        cur_row = 0
        for i, row in cur_df.iterrows():

            # print(f'\n\tROW {cur_row+1} of {n_rows}')

            cur_label = row[interest_cols['label']] # get the label of the row

            # print(f'\n\tTHIS IS THE CURRENT ANNOTATION LABEL:\n\t{cur_label}')

            # get the color (BGR) for the label
            cur_col = label_bgrs[cur_label]

            # print(f'\n\tTHIS IS THE BGR FOR THE LABEL:\n\t{cur_col}')

            # make rectangle xy coordinates
            x1 = row[interest_cols['x1']]
            y1 = row[interest_cols['y1']]
            x2 = row[interest_cols['x2']]
            y2 = row[interest_cols['y2']]
            coord_1, coord_2 = (x1, y1), (x2, y2)

            # print(f'\n\tTHIS IS THE TOP LEFT COORDINATE:\n\t{coord_1}')
            # print(f'\n\tTHIS IS THE BOTTOM RIGHT COORDINATE:\n\t{coord_2}')

            # draw the bounding box
            cv.rectangle(cur_img, 
                         coord_1, 
                         coord_2, 
                         cur_col, 
                         thickness=line_thickness, 
                         lineType=cv.LINE_AA,
                        )

            # print(f'\n\tTHIS IS THE IMAGE WITH ADDED BBOX:\n\t')
            # plt.imshow(cv.cvtColor(cur_img, cv.COLOR_BGR2RGB))
            # plt.show()

            # insert text label
            font_thickness = max(line_thickness - 1, 1)
            w, h = cv.getTextSize(cur_label, 
                                  0, 
                                  fontScale=line_thickness / 3, 
                                  thickness=font_thickness)[0]
            coord_2 = coord_1[0] + w, coord_1[1] - h - 3
            cv.rectangle(cur_img, 
                         coord_1, 
                         coord_2, 
                         cur_col, 
                         -1, 
                         cv.LINE_AA,
                        )
            cv.putText(cur_img, 
                       cur_label, 
                       (coord_1[0], coord_1[1] - 2), 
                       0, 
                       line_thickness / 3, 
                       txt_color, 
                       thickness=font_thickness, 
                       lineType=cv.LINE_AA,
                      )

            # print(f'\n\tTHIS IS THE IMAGE WITH ADDED LABEL TEXT:\n\t')
            # plt.imshow(cv.cvtColor(cur_img, cv.COLOR_BGR2RGB))
            # plt.show()

            cur_row += 1


        out_filepath = folder_locs['out_pics'] + filename # create relative filepath of the image to be saved

        # print(f'\nSAVING THE FILE TO HERE:\n{out_filepath}')

        cv.imwrite(out_filepath, cur_img) # save the file
        
    except:
        pass
    